In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

In [2]:
input_url = "https://www.tripadvisor.com/Tourism-g304554-Mumbai_Maharashtra-Vacations.html"
num_attractions_per_city = 100
num_reviews_per_attraction = 100
city_name = input_url[input_url.find("Tourism-g")+16:input_url.find("-Vacations")]

In [ ]:
city_code = input_url[input_url.find("Tourism-g")+8:input_url.find("-" + city_name)]

city_url = "https://www.tripadvisor.in/Attractions-" + city_code + "-Activities-oa0-" +city_name + ".html"
domain = city_url[:city_url.find("Attractions-g")-1]

city_url1 = city_url[:city_url.find("Activities-oa")+13]
city_url2 = city_url[city_url.find("Activities-oa")+14:]

city_urls = [(city_url1 + str(i) + city_url2) for i in list(map(lambda x: 30 * x, range(1, 1000)))]
city_urls.insert(0, city_url)

attraction_urls = []

for ind, url in enumerate(city_urls):
    page = requests.get(url, headers={'User-Agent': "Mozilla/5.0"})
    soup = BeautifulSoup(page.text, 'html.parser')
    d = soup.find("div", class_="DDJze")
    try:
        att_urls_on_page = d.find_all("div", class_="alPVI eNNhq PgLKC tnGGX")
        length = len(att_urls_on_page)
        if (length == 0):
            break
        for i in range(length):
            attraction_urls.append(domain + att_urls_on_page[i].find_all('a')[0].get('href'))
    except:
        pass
    print(ind)
        
city_attraction_urls = pd.DataFrame(attraction_urls, columns = ['attraction_urls'])
city_attraction_urls.to_csv(city_name + "_Attraction_Urls.csv")

In [ ]:
city_attraction_urls = pd.read_csv(city_name + "_Attraction_Urls.csv")

attraction_number = 0

for attraction_url in city_attraction_urls.attraction_urls:
    
    attraction_name = attraction_url[attraction_url.find("Reviews")+8:attraction_url.find(city_name)-1]
    print(attraction_number, "  ", attraction_name)
    
    try:
        pd.read_csv("Individual Attractions/" + attraction_name + "_" + city_neme + ".csv")
    except:

        attraction_url1 = attraction_url[:attraction_url.find("Reviews")+8]+"or"
        attraction_url2 = attraction_url[attraction_url.find("Reviews")+7:]

        review_urls = [(attraction_url1 + str(i) + attraction_url2) for i in list(map(lambda x: 10 * x, range(1, 30)))]
        review_urls.insert(0, attraction_url)
        
        review_page_number = 0
        
        for ind, url in enumerate(review_urls):
            page = requests.get(url, headers={'User-Agent': "Mozilla/5.0"})
            soup = BeautifulSoup(page.text, 'html.parser')
                    
            attraction = []
            name_of_reviewer = []
            reviewer_rating = []
            review_date = []
            review_title = []
            review_text = []
            
            d = soup.find("div", class_="LbPSX")
            reviewer_names = d.find_all("div", class_="zpDvc")
            reviewer_ratings =  d.find_all("svg", class_="UctUV d H0")
            review_titles = d.find_all("div", class_="biGQs _P fiohW qWPrE ncFvv fOtGX")
            review_dates = d.find_all("div", class_="RpeCd")
            review_texts =  d.find_all("div", class_="biGQs _P pZUbB KxBGd")
            
            length = len(reviewer_names)
            length1 = len(reviewer_ratings)
            length2 = len(review_dates)
            length3 = len(review_titles)
            length4 = len(review_texts)
            
            if (length == 0):
                break
            try:
                for i in range(length):
                    name_of_reviewer.append(reviewer_names[i].contents[0].contents[0].contents[0])
                    attraction.append(attraction_name)
                for i in range(length1):
                    reviewer_rating.append(reviewer_ratings[i]['aria-label'])
                for i in range(length2):      
                    review_date.append(review_dates[i].contents[0])
                for i in range(length3):
                    review_title.append(review_titles[i].contents[0].contents[0].text)
                for i in range(length4):
                    review_text.append(review_texts[i].contents[0].text)
            except:
                break


            dic = {}
            dic["Tourist Attraction"] = attraction
            dic['Name'] = name_of_reviewer
            dic['Rating'] = reviewer_rating
            dic['Date and criteria'] = review_date
            dic['Title'] = review_title
            dic['Review'] = review_text      
                
            reviews_on_page = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in dic.items() ]))
            
            if (review_page_number==0):
                attraction_reviews  = reviews_on_page[:]
            else:
                attraction_reviews = pd.concat([attraction_reviews,reviews_on_page])
            
            review_page_number = review_page_number+1

        if (attraction_number==0):
            city_reviews = attraction_reviews[:]
        else:
            city_reviews = pd.concat([city_reviews,attraction_reviews])
        
        attraction_reviews.to_csv("Individual Attractions/" + attraction_name + "_" + city_name + ".csv")
    
    attraction_number = attraction_number + 1
    
    city_reviews.to_csv(city_name + "_Attraction_Reviews.csv")
    
city_reviews.to_csv(city_name + "_Attraction_Reviews.csv")

In [10]:
city_attraction_urls = pd.read_csv(city_name + "_Attraction_Urls.csv")

attraction_number = 0

for attraction_url in city_attraction_urls.attraction_urls:
    
    attraction_name = attraction_url[attraction_url.find("Reviews")+8:attraction_url.find(city_name)-1]
    print(attraction_number, "  ", attraction_name)
    
    try:
        pd.read_csv("Limited Individual Attractions/" + attraction_name + "_" + city_name + ".csv")
        city_reviews = pd.read_csv(city_name + "_Limited_Attraction_Reviews.csv")
    except:

        attraction_url1 = attraction_url[:attraction_url.find("Reviews")+8]+"or"
        attraction_url2 = attraction_url[attraction_url.find("Reviews")+7:]

        review_urls = [(attraction_url1 + str(i) + attraction_url2) for i in list(map(lambda x: 10 * x, range(1, 30)))]
        review_urls.insert(0, attraction_url)

        review_page_number = 0
        
        for ind, url in enumerate(review_urls):
            
            try:
                page = requests.get(url, headers={'User-Agent': "Mozilla/5.0"})
                soup = BeautifulSoup(page.text, 'html.parser')
                        
                attraction = []
                name_of_reviewer = []
                reviewer_rating = []
                review_date = []
                review_title = []
                review_text = []
                
                d = soup.find("div", class_="LbPSX")
                reviewer_names = d.find_all("div", class_="zpDvc")
                reviewer_ratings =  d.find_all("svg", class_="UctUV d H0")
                review_titles = d.find_all("div", class_="biGQs _P fiohW qWPrE ncFvv fOtGX")
                review_dates = d.find_all("div", class_="RpeCd")
                review_texts =  d.find_all("div", class_="biGQs _P pZUbB KxBGd")
                
                length = len(reviewer_names)
                length1 = len(reviewer_ratings)
                length2 = len(review_dates)
                length3 = len(review_titles)
                length4 = len(review_texts)
                
                if (length == 0):
                    break
                try:
                    for i in range(length):
                        name_of_reviewer.append(reviewer_names[i].contents[0].contents[0].contents[0])
                        attraction.append(attraction_name)
                    for i in range(length1):
                        reviewer_rating.append(reviewer_ratings[i]['aria-label'])
                    for i in range(length2):      
                        review_date.append(review_dates[i].contents[0])
                    for i in range(length3):
                        review_title.append(review_titles[i].contents[0].contents[0].text)
                    for i in range(length4):
                        review_text.append(review_texts[i].contents[0].text)
                except:
                    break

                dic = {}
                dic["Tourist Attraction"] = attraction
                dic['Name'] = name_of_reviewer
                dic['Rating'] = reviewer_rating
                dic['Date and criteria'] = review_date
                dic['Title'] = review_title
                dic['Review'] = review_text      
                    
                reviews_on_page = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in dic.items() ]))
                
                if (review_page_number==0):
                    attraction_reviews  = reviews_on_page[:]
                else:
                    attraction_reviews = pd.concat([attraction_reviews,reviews_on_page])
                
                review_page_number = review_page_number+1
                    
                if len(attraction_reviews) >= num_reviews_per_attraction:
                    break
            except:
                pass

        try:
            city_reviews = pd.concat([city_reviews,attraction_reviews])
        except:
            city_reviews = attraction_reviews[:]
        
        attraction_reviews.to_csv("Limited Individual Attractions/" + attraction_name + "_" + city_name + ".csv", index = False)
    
    attraction_number = attraction_number + 1
    
    city_reviews.to_csv(city_name + "_Limited_Attraction_Reviews.csv", index = False)
    
    if attraction_number == num_attractions_per_city:
        break
    
city_reviews.to_csv(city_name + "_Limited_Attraction_Reviews.csv", index = False)

FileNotFoundError: [Errno 2] No such file or directory: 'Mumbai_Maharashtra_Attraction_Urls.csv'

In [20]:
#cd Limited Individual Attractions
import os
cd ..

SyntaxError: invalid syntax (3406706207.py, line 3)

In [ ]:
from os import listdir
files = listdir()
limited_2 = pd.read_csv(files[0])
for file_name in files:
    limited_2.append(pd.read_csv(file_name))
limited_2.to_csv("Check.csv")